# Concatenate videos
This notebook takes care of concatenating a bunch of .tif files with doric recordings from an experiment. Additionally it will do flat field subtraction from the concatenated video.
It will was both the raw and ffsub videos as .tif files

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from os import listdir
from os.path import join, splitext
import cv2
import re
import caiman as cm

## Get list of files
Define the base directory that contains your source files. We use *Regular Expressions* to filter the files within this directory: only files ending in '_dX.tif' (where X is a number) will be processed. This pattern should match your full resolution files coming from the camera. We'll also want to correctly sort the list of files.

In [ ]:
# Get the videos in the data folder
data_fld = "D:\\Dropbox (UCL - SWC)\\Project_vgatPAG\\analysis\\doric\\BF164p1\\19JUN05"

# TODO add ouput folder
raise NotImplementedError

regex     = re.compile(r'.*_d\d+.tif$')
files_in  = list(filter(regex.search, listdir(data_fld)))
files_in.sort(key=lambda x: int(re.search('.*_d(\d+?).tif$', x).group(1)))
print(*files_in, sep = "\n") 

In [ ]:
data_fld = "D:\\Dropbox (UCL - SWC)\\Project_vgatPAG\\analysis\\doric\\BF164p2\\19JUN26"

files_in = ['19JUN26_BF164p2_fullres.tif']

## Load, scale and concatenate files


In [ ]:
scale     = .5 # normally it's .2
framerate = 10
movies    = []
for f in files_in:
    movies.append(cm.load(join(data_fld,f), fr=framerate).resize(scale,scale,1))
movie     = cm.concatenate(movies, axis=0)

# start and end framenumbers for later use
frames1   = np.cumsum([m.shape[0] for m in movies])
frames0   = np.append(0,frames1[0:-1]) + 1

# free some memory ...
del movies

## Flat-field correction
To obtain an estimate of the movie's flat-field, we first calculate the average intensity across time. We then apply a gaussian blur to the resulting image. Finally, the flat-field is centered around zero by substracting its mean.

The flat-field image will be *substracted* from all frames of the movie.

Refer to: https://imagej.net/Image_Intensity_Processing#Pseudo-correction

In [ ]:
sigma      = 75, # was 25 for downsampled movies
sigma      = int(np.ceil(sigma) // 2 * 2 + 1)
average    = np.mean(movie, axis=0)
flat_field = cv2.GaussianBlur(average,(sigma,sigma),0)

norm_by = "subtraction"

if norm_by == "subtraction":
    '''movie_fcc by subtraction'''
    flat_field = flat_field - flat_field.mean()
    movie_fcc  = movie - flat_field
elif norm_by == "division":
    '''movie_fcc by division'''
    movie_fcc  = movie / flat_field
    movie_fcc  = movie_fcc * movie.mean()
else:
    raise ValueError("Invalid norm by argument")


# Visualise the effect of the normalization
fig, axes  = plt.subplots(nrows=1, ncols=3, figsize=(15,15))
axes[0].imshow(movie[1], cmap='gray', vmin=movie[1].min(), vmax=movie[1].max())
axes[0].set_title('first frame')
axes[1].imshow(flat_field, cmap='gray')
axes[1].set_title('flat-field')
axes[2].imshow(movie_fcc[1], cmap='gray', vmin=movie[1].min(), vmax=movie[1].max())
axes[2].set_title('first frame (flat-field corrected)')

In [ ]:
movie_fcc.save(join(data_fld, '19JUN26_BF164p2_halfres_ffcSub' + '.tif'))


## Save processed movie to TIF, write indices to npy

Save the concatenated videos and the length of each original video 

In [ ]:
file_out = re.search('(.*?)_d\d+.tif$', files_in[1]).group(1) + '_ds{}_ffc'.format(movie.shape[1])
movie_fcc.save(join(data_fld,file_out + '.tif'))
print(f"Saving ffcsub video at: {join(dir_base,file_out + '.tif')}")


file_out2 = re.search('(.*?)_d\d+.tif$', files_in[1]).group(1) + '_ds{}_raw'.format(movie.shape[1])
movie.save(join(data_fld,file_out2 + '.tif'))
print(f"Saving raw video at: {join(dir_base,file_out2 + '.tif')}")

#save tiff lengths
np.save(join(data_fld,file_out + '_tifflengths.npy'),frames1)